In [5]:
import pandas as pd
import numpy as np
import re
import string

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D, TextVectorization, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model, load_model

from wordcloud import WordCloud, STOPWORDS
stop_words = stopwords.words('english')

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# load model
load_model_func = load_model('sentiment.h5')

In [10]:
text = "app look shitty x change back twitter"
data = {
    'text': [text],
}
tokenizer = Tokenizer()
df_x = pd.DataFrame(data)

In [11]:
x_data_val = pad_sequences(tokenizer.texts_to_sequences(df_x.text), maxlen = 30)

In [12]:
x_scores = load_model_func.predict(x_data_val, verbose = 1, batch_size = 10000)
x_scores[:2]

1/1 [==============================] - 1s 655ms/step


array([[0.02769713, 0.937194  , 0.03510877]], dtype=float32)

In [13]:
df2 = pd.DataFrame(x_scores, columns=['Positive', 'Negative', 'Neutral'])
new_df2 = pd.DataFrame(np.where(x_scores == np.max(x_scores, axis=1, keepdims=True), True, False), columns=['Positive_class', 'Negative_class', 'Neutral_class'])
df2 = pd.concat([df2, new_df2], axis=1)
condf2 = pd.concat([df_x, df2], axis=1)
condf2.head()

,text,Positive,Negative,Neutral,Positive_class,Negative_class,Neutral_class
0,app look shitty x change back twitter,0.027697,0.937194,0.035109,False,True,False


In [14]:
condf2['Sentiment'] = condf2[['Positive_class', 'Negative_class', 'Neutral_class']].idxmax(axis=1)
condf2['Sentiment'] = condf2['Sentiment'].str.replace('_class', '')
condf2

,text,Positive,Negative,Neutral,Positive_class,Negative_class,Neutral_class,Sentiment
0,app look shitty x change back twitter,0.027697,0.937194,0.035109,False,True,False,Negative
